In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pickle
import numpy as np
from matplotlib import pyplot as plt
import csv
import tensorflow as tf
import random
from sklearn.utils import shuffle
import cv2


def LeNet(x, drop_rate=0.5):
    # For normalization of the input features
    mu = 0
    sigma = 0.1

    # Convolution 1 + Activation + Pooling
    conv_1_w = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 16), mean=mu, stddev=sigma))
    conv_1_b = tf.Variable(tf.zeros(16))
    conv_1 = tf.nn.conv2d(x, conv_1_w, strides=[1, 1, 1, 1], padding='VALID') + conv_1_b  # Conv
    conv_1 = tf.nn.relu(conv_1)  # Activation
    conv_1 = tf.nn.max_pool(conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')  # Pooling

    # Convolution 2 + Activation + Pooling
    conv_2_w = tf.Variable(tf.truncated_normal(shape=(5, 5, 16, 32), mean=mu, stddev=sigma))
    conv_2_b = tf.Variable(tf.zeros(32))
    conv_2 = tf.nn.conv2d(conv_1, conv_2_w, strides=[1, 1, 1, 1], padding='VALID') + conv_2_b  # Conv
    conv_2 = tf.nn.relu(conv_2)  # Activation
    conv_2 = tf.nn.max_pool(conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')  # Pooling
    
    # Flatten the last conv layer
    fc_0 = tf.contrib.layers.flatten(conv_2)

    # Layer 3
    fc3_W = tf.Variable(tf.truncated_normal(shape=(800, 500), mean = mu, stddev = sigma))
    fc3_b = tf.Variable(tf.zeros(500))
    fc3   = tf.matmul(fc_0, fc3_W) + fc3_b
    fc3 = tf.nn.relu(fc3)
    fc3 = tf.nn.dropout(fc3, rate=drop_rate)

    # Layer 4
    fc4_W  = tf.Variable(tf.truncated_normal(shape=(500, 100), mean = mu, stddev = sigma))
    fc4_b  = tf.Variable(tf.zeros(100))
    fc4 = tf.matmul(fc3, fc4_W) + fc4_b
    fc4 = tf.nn.dropout(fc4, rate=drop_rate)

    # Layer 5
    fc5_W  = tf.Variable(tf.truncated_normal(shape=(100, 43), mean = mu, stddev = sigma))
    fc5_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc4, fc5_W) + fc5_b

    
    return logits, ([conv_1_w, conv_2_w, fc3_W, fc4_W, fc5_W])


# Load the Image data
training_file = '/content/drive/My Drive/P3-Traffic_Sign_Classifier/traffic-signs-data/train.p'
validation_file = '/content/drive/My Drive/P3-Traffic_Sign_Classifier/traffic-signs-data/valid.p'
testing_file = '/content/drive/My Drive/P3-Traffic_Sign_Classifier/traffic-signs-data/test.p'


with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']


# # Convert to gray scale
# new_data = []
# for data in X_train:
#     image = data[0:]
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     gray = np.reshape(gray, (32, 32, 1))
#     new_data.append(gray)
# X_train = np.array(new_data)
# print(X_train.shape)

# new_data = []
# for data in X_valid:
#     image = data[0:]
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     gray = np.reshape(gray, (32, 32, 1))
#     new_data.append(gray)
# X_valid = np.array(new_data)

# new_data = []
# for data in X_test:
#     image = data[0:]
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     gray = np.reshape(gray, (32, 32, 1))
#     new_data.append(gray)
# X_test = np.array(new_data)

# Normalize
X_train = (X_train.astype(float))/float(255)  # Normalize the data between 0-1
X_valid = (X_valid.astype(float))/float(255)

# Load the cvs file into a dict
csv_file = '/content/drive/My Drive/P3-Traffic_Sign_Classifier/signnames.csv'
sign_lable_names = {}
line_no = 0
with open(csv_file, mode='r') as f:
    csv_data = csv.reader(f, delimiter=',')
    for line in csv_data:
        if line_no == 0:
            line_no = 1
            continue
        else:
            sign_lable_names[int(line[0])] = line[1]



# Get the different sizes
# Number of training examples
n_train = y_train.shape[0]

# Number of validation examples
n_validation = y_valid.shape[0]

# Number of testing examples.
n_test = y_test.shape[0]

# What's the shape of an traffic sign image?
image_shape = str(X_test[0].shape[1]) + 'x' + str(X_test[0].shape[0])

# How many unique classes/labels there are in the dataset.
n_classes = len(sign_lable_names.keys())

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
print("Image Shape = ", X_test[0].shape)
print("Input Shape =", X_train.shape)
print("Output Shape =", y_train.shape)

# Hyperparameters to tune
EPOCHS = 250
BATCH_SIZE = 8
RATE = 0.00001
DROP_RATE = 0.7
L2_SCALE = 1e-3

# Create Placeholders
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
drop_rate = tf.placeholder(tf.float32, ())
one_hot_y = tf.one_hot(y, n_classes)

# Training pipeline
logits, weights = LeNet(x, drop_rate)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)

# Loss function using L2 Regularization
l2_regularizer = 0.0
for weight in weights:
    l2_regularizer += tf.nn.l2_loss(weight)
loss_operation = tf.reduce_mean(cross_entropy)
loss_operation = tf.reduce_mean(loss_operation + L2_SCALE * l2_regularizer)

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=RATE)
training_operation = optimizer.minimize(loss_operation)


# Evaluation the model
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(x_data, y_data):
    num_examples = len(x_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = x_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, drop_rate:0.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

# Train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, drop_rate:DROP_RATE})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        training_accuracy = evaluate(X_train, y_train)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

# Evaluate the model with the test images
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    
    validation_accuracy = evaluate(X_valid, y_valid)
    print("Validation Accuracy = {:.3f}".format(validation_accuracy))

Number of training examples = 34799
Number of testing examples = 12630
Image data shape = 32x32
Number of classes = 43
Image Shape =  (32, 32, 3)
Input Shape = (34799, 32, 32, 3)
Output Shape = (34799,)
Training...

EPOCH 1 ...
Training Accuracy = 0.206
Validation Accuracy = 0.201

EPOCH 2 ...
Training Accuracy = 0.298
Validation Accuracy = 0.273

EPOCH 3 ...
Training Accuracy = 0.367
Validation Accuracy = 0.322

EPOCH 4 ...
Training Accuracy = 0.431
Validation Accuracy = 0.377

EPOCH 5 ...
Training Accuracy = 0.472
Validation Accuracy = 0.405

EPOCH 6 ...
Training Accuracy = 0.504
Validation Accuracy = 0.435

EPOCH 7 ...
Training Accuracy = 0.526
Validation Accuracy = 0.453

EPOCH 8 ...
Training Accuracy = 0.545
Validation Accuracy = 0.475

EPOCH 9 ...
Training Accuracy = 0.567
Validation Accuracy = 0.499

EPOCH 10 ...
Training Accuracy = 0.589
Validation Accuracy = 0.524

EPOCH 11 ...
Training Accuracy = 0.610
Validation Accuracy = 0.543

EPOCH 12 ...
Training Accuracy = 0.638
Valida

W0813 20:23:12.565088 139897215133568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


EPOCH 250 ...
Training Accuracy = 0.999
Validation Accuracy = 0.962

Model saved
Test Accuracy = 0.941
Validation Accuracy = 0.962
